In [38]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import time
from threading import Thread, Event
import pandas as pd
from typing import List
import datetime

In [39]:
def get_date_range_for_season(year):
    switcher = {
        2020: ["october-2019", "november", "december", "january", "february", "march", "july", "august", "september", "october-2020"],
        2019: pd.date_range('2018-11-01', '2019-3-31'),
        2018: pd.date_range('2017-11-01', '2018-3-31'),
        2017: pd.date_range('2016-11-01', '2017-3-31'),
        2016: pd.date_range('2015-11-01', '2016-3-31'),
        2015: pd.date_range('2014-11-01', '2015-3-31'),
        2014: pd.date_range('2013-11-01', '2014-3-31'),
        2013: pd.date_range('2012-11-01', '2013-3-31'),
        2012: pd.date_range('2012-1-01', '2012-3-31'),
        2011: pd.date_range('2010-11-01', '2011-3-31'),
        2010: pd.date_range('2009-11-01', '2010-3-31')
    }
    return switcher.get(year, "out of range... range is 2010-2020")

In [40]:
class DataHolder():
    def __init__(self, stat_url_endpoints):
        self.stat_dict = {}
        self.keys = stat_url_endpoints
        for key in self.keys:
            self.stat_dict[key] = []

    def print(self):
        print(self.stat_dict)
    
    def getDataDict(self):
        return self.stat_dict

In [41]:
def scrape_stat_data(url, daterange):
    print(f"going to {url} starting at: {daterange[0].date()}")
    data_array = []
    for date in daterange:
        if date == pd.Timestamp('2013-12-31'): # skip this date there is no data on the website...
            continue
        response = requests.get(url + date.strftime("%Y-%m-%d"))
        data = [row.find_all('td')[2].text for row in BeautifulSoup(response.text, 'html.parser').find_all('tr')[1:]] # changed to '3' to get the 'last 3 games data' on the website
        teams = [team.text for team in BeautifulSoup(response.text, 'html.parser').find_all('td', class_='text-left nowrap')]
        zip_list = list(zip(*[teams, data]))
        print(date.date())
        zip_list.sort()
        teams_list, data_list = zip(*zip_list)
        if '%' in data_list[0]: # handles the percent symbol in certain data points
            data_list = [float(x[0:-1]) for x in data_list]
        else:
            data_list = [float(x) for x in data_list]
        # data_array.append([[date for i in range(30)], list(zip(*[teams, pts]))])
        data_array.append([[date for i in range(30)], teams_list, data_list])
    return data_array

In [42]:
def send_scrape_threads(DHolder: DataHolder, years):
    dateRanges = [get_date_range_for_season(year) for year in years]
    print(dateRanges)
    numDateRanges = len(dateRanges)
    print(numDateRanges)
    for endpoint in DHolder.keys: 
        url = f"https://www.teamrankings.com/nba/stat/{endpoint}?date="
        data_list = []
        # ppg threads
        t0 = time.time()
        with concurrent.futures.ThreadPoolExecutor(max_workers=numDateRanges) as executor:
            results = executor.map(scrape_stat_data, [url]*numDateRanges, dateRanges)
        for val in results:
            print(f"val: {val[0]} \n")  
            data_list += val
            # print(val[0])
        t1 = time.time()
        DHolder.stat_dict[endpoint] = data_list
        # DHolder.print()
        print(f"this took {round(t1-t0,2)} seconds USING {numDateRanges} WORKERS!!!.")

In [43]:
def get_team_data(startYear: int, endYear: int, endpoints: List[str]):
    DHolder = DataHolder(endpoints)
    years = [x + startYear for x in range(endYear-startYear)]
    print(years)
    # print(years) # loops over different stats
    send_scrape_threads(DHolder, years)
    
    keys = DHolder.keys
    number_of_days = len(DHolder.stat_dict[keys[0]])
    print(number_of_days)
    date_index, team_index =  [], []
    for i in range(number_of_days):
        date_index += DHolder.stat_dict[keys[0]][i][0]
        team_index += DHolder.stat_dict[keys[0]][i][1]
    # get DHolder.stat_dict ready to be made into a data frame
    # turn the original list into a long list that holds JUST the data for that endpoint. It will be in the proper order since the teams were sorted (see zipping) in scrape_stat_data. By doing this we will bea able to make a dataframa out of Dholder.stat_dict.
    for key in keys:
        data_list = []
        for j in range (number_of_days):
            data_list += DHolder.stat_dict[key][j][2] # this j is important 
        DHolder.stat_dict[key] = data_list # replace list containing team and date info with only the stat list(datalist)

    DF =  pd.DataFrame(data=DHolder.stat_dict, index=[date_index, team_index])
    return DF

In [44]:
t0 = time.time()
endpoints = ['true-shooting-percentage', 'defensive-efficiency', 'average-margin-thru-3-quarters', 'opponent-4th-quarter-points-per-game', 'offensive-efficiency', 'opponent-shooting-pct', 'average-scoring-margin', 'opponent-defensive-rebounding-pct', 'opponent-offensive-rebounding-pct' , 'defensive-rebounding-pct', 'offensive-rebounding-pct' ]
DF = get_team_data(2015, 2020, endpoints) # 2015
t1 = time.time()
secs = t1-t0
time_taken = str(datetime.timedelta(seconds=secs))
print(f"The time for {len(endpoints)} endpoints was {time_taken}")

018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), Timestamp('2018-11-01 00:00:00', freq='D'), 

In [45]:
DF.to_pickle('data_df_2015-2020-full-season.pickle')
# The above has the following end points for the whole season
# endpoints = ['true-shooting-percentage', 'defensive-efficiency', 'average-margin-thru-3-quarters', 'opponent-4th-quarter-points-per-game', 'offensive-efficiency', # #'opponent-shooting-pct', 'average-scoring-margin', 'opponent-defensive-rebounding-pct', 'opponent-offensive-rebounding-pct' , 'defensive-rebounding-pct', # #'offensive-rebounding-pct' ]

In [33]:
# DF.to_pickle('data_df_2015-2020-Last3.pickle') TOOK 0:46:36.133975 to scrape this :0
# The above DF has these endpoints 
# endpoints = ['true-shooting-percentage', 'defensive-efficiency', 'average-margin-thru-3-quarters', 'opponent-4th-quarter-points-per-game', 'offensive-efficiency', 'opponent-shooting-pct', 'average-scoring-margin', 'opponent-defensive-rebounding-pct', 'opponent-offensive-rebounding-pct' , 'defensive-rebounding-pct', 'offensive-rebounding-pct' ]

In [32]:
DF

true-shooting-percentage  defensive-efficiency  \
2014-11-01 Atlanta                         115.8                 1.108   
           Boston                          122.3                 1.013   
           Brooklyn                        113.7                 1.168   
           Charlotte                        94.1                 0.981   
           Chicago                         115.6                 0.970   
...                                          ...                   ...   
2019-03-31 Sacramento                      114.7                 1.194   
           San Antonio                     110.9                 1.063   
           Toronto                         121.5                 0.950   
           Utah                            125.8                 1.007   
           Washington                      110.0                 1.168   

                        average-margin-thru-3-quarters  \
2014-11-01 Atlanta                               -15.0   
           Boston                                 29.0   
           Brooklyn                              -29.0   
           Charlotte                             -15.0   
           Chicago                                 7.5   
...                                                ...   
2019-03-31 Sacramento                             -1.7   
           San Antonio                             4.3   
           Toronto                                18.0   
           Utah                                   10.7   
           Washington                             -5.7   

                        opponent-4th-quarter-points-per-game  \
2014-11-01 Atlanta                                      23.0   
           Boston                                       33.0   
           Brooklyn                                     20.0   
           Charlotte                                    17.0   
           Chicago                                      20.0   
...                                                      ...   
2019-03-31 Sacramento                                   29.3   
           San Antonio                                  25.7   
           Toronto                                      28.3   
           Utah                                         27.0   
           Washington                                   31.3   

                        offensive-efficiency  opponent-shooting-pct  \
2014-11-01 Atlanta                     1.037                   41.1   
           Boston                      1.168                   48.8   
           Brooklyn                    1.013                   55.7   
           Charlotte                   0.999                   48.8   
           Chicago                     1.060                   38.1   
...                                      ...                    ...   
2019-03-31 Sacramento                  1.161                   49.6   
           San Antonio                 1.115                   45.1   
           Toronto                     1.133                   41.4   
           Utah                        1.172                   42.8   
           Washington                  1.109                   52.9   

                        average-scoring-margin  \
2014-11-01 Atlanta                        -7.0   
           Boston                         16.0   
           Brooklyn                      -16.0   
           Charlotte                       2.0   
           Chicago                         9.0   
...                                        ...   
2019-03-31 Sacramento                     -3.3   
           San Antonio                     5.3   
           Toronto                        19.0   
           Utah                           17.3   
           Washington                     -6.3   

                        opponent-defensive-rebounding-pct  \
2014-11-01 Atlanta                                   76.2   
           Boston                                    76.9   
           Brooklyn             